# seq2seq模型之自动聊天机器人
本章主要介绍如何将seq2seq模型转换为PyTorch可用的前端混合Torch脚本。我们要转换的模型来自于聊天机器人教程，[click here](https://pytorch.org/tutorials/beginner/chatbot_tutorial.html)。

## 混合前端
在一个基于深度学习项目的研发阶段，使用像PyTorch这样即时 eager 、命令式的页面进行交互能带来很大的便利。这使用户能够在使用Python数据结构、控制流操作、打印语句和调用程序时通过熟悉的、管用的Python脚本编写。  

尽管即时性界面对于研究和试验应用程序时一个有用的工具，但是对于生产环境中部署模型时，使用基于图像 graph-based 的模型表示将更加适用。一个延迟的图型展示意味着可以优化，比如无序执行操作，以及针对高度优化的硬件架构的能力。此外，基于图像的表示支持框架无关的模型导出。**PyTorch提供了将即时模式的代码增量转换为Torch脚本的机制**，Torch脚本是一个在Python中的静态可分析和可优化的子集，Torch使用它来在Python运行时独立进行深度学习。  

**在Torch中的torch.jit模块可以找到即时模式的PyTorch转换为Torch脚本的API**。这个模块有两个核心模式用于将即时模式模型转换为Torch的脚本图形表示；**跟踪 tracing 以及脚本化 scripting. torch.jit.trace 函数接收一个模块或者一个函数和一组示例的输入, 然后通过函数或者模块运行输入示例**，同时跟踪遇到的计算步骤，然后输出一个可以展示跟踪流程的基于图形的函数。 跟踪 Tracing 对于不涉及依赖于数据的控制流的直接的模块和函数非常有用，就比如标准的卷积神经网络。  

然而，如果一个有数据依赖的if语句和循环函数被跟踪，则只记录示例输入沿执行路径调用的操作。换句话说，控制流本身并没有被捕获。要将带有数据依赖的控制流的模块和函数进行转换，已提供了一个脚本化机制。基本显式地 将模块或函数代码转换为Torch脚本，包括所有可能的控制流路径。如需使用脚本模式script mode，要确定继承了torch.jit.ScriptMoudle基本类(取代torch.nn.Moudle)并且增加torch.jit.script装饰器到你的Python函数或者torch.jit.script_method装饰器到你的模块方法。  

使用脚本化的一个警告是，它只支持Python的一个受限子集。要获取与支持的特性相关的所有详细信息，请参考Torch Script language reference。 为了达到最大的灵活性，可以组合Torch脚本的模型来表示整个程序，并且可以增量地应用这些技术。  

下图是一个混合前端的工作流示例图：  

![figure.1](https://gitee.com/zyp521/upload_image/raw/master/Dy2Inx.png)

## 预备环境
首先，导入所需的模块以及设置一些常量。如果想使用自己的模型，需要保证MAX_LENGTH 常量设置正确。提醒：这个常量定义了在训练过程中允许的最大句子长度以及模型能够产生的最大句子长度。

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import os
import unicodedata
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_LENGTH = 10 # 最大句子长度

# 默认的词向量
PAD_token = 0 # 用于对句子进行空白填充
SOS_token = 1 # 句子开头的符号
EOS_token = 2 # 句子结尾的符号

## 模型概述
正如前文所说的，我么使用 [sequence-to-sequence](https://arxiv.org/abs/1409.3215) 模型。这种类型的模型用于输入是可变长度序列的情况，我们的输出也是一个可变长度的序列，它不一定是一对一输入映射。seq2seq 模型由两个递归神经网络（RNNs）组成：**编码器和解码器**。  

![figure.2](https://gitee.com/zyp521/upload_image/raw/master/OkxwwE.png)

### 编码器（Encoder）
编码器RNN在输入语句中每次迭代一个标记（例如单词）每次步骤输出一个“输出”向量和一个隐藏状态“向量”。“隐藏状态”向量在之后则传递到下一个步骤，同时记录输出向量。编码器将序列中每个坐标代表的文本转换为月高位空间的中的一组坐标，解码器将使用这些坐标为给定的任务生成有意义的输出。

### 编码器
解码器RNN以逐个令牌的方式生成响应语句。它使用来自于编码器的文本向量和内容隐藏状态来序列中的下一个单词。它继续生成单词，直到输出表示句子结束的EOS语句。我们在解码器中使用专注机制[Attention Mechanism](https://arxiv.org/abs/1409.0473)来帮助它在输入的某些部分生成输出时“保持专注”。对于我们的模型，我们实现了 Luong et al等人的“全局关注 Goal attention”模块，**并将其作为解码器中的子模块**。

## 数据处理
尽管我们的模型在概念上处理标记序列，但在现实中，它们与所有机器学习模型一样处理数字。在这种情况下，**训练之前建立的模型词汇表中的每个单词都映射到一个整数索引**。我们使用Voc对象来包含从单词到索引的映射，以及词汇表中的单词总数。我们将在运行模型之前加载对象。  

此外，**为了能够进行评估，我们必须提供一个处理字符串输入的工具。 normalizeString 函数将字符串中的所有字符转换为小写，并删除所有非字母的字符。 indexesFromSentence 函数接收一个单词的句子并返回相应的单词索引序列**。

In [70]:
class Voc:
    # 初始化
    def __init__(self,name):
        self.name = name
        self.trimmed = False
        # 映射词典分别为单词对应标号、单词词频、标号对应单词
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:"PAD",SOS_token:'SOS',EOS_token:"EOS"}
        self.num_words = 3 # 统计词汇表单词数量 基础为3，分别为PAD SOS EOS
    
    # 将句子中的单词添加到词汇表中
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words # 赋值给新单词的序列
            self.word2count[word] = 1 # 新单词出现次数为1
            self.index2word[self.num_words] = word
            self.num_words += 1
        
        else:
            self.word2count[word] += 1 # 如果是已经存在的单词，则词频+1
    
    # 统计大于等于最小值词频的单词，并在此基础上创建新的词典 trimm平衡 threshold临界值、阈值
    def trim(self, mini_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        for k,v in self.word2count.items():
            if v >= mini_count:
                keep_words.append(k)
        
        print('keep_words {} / {} = {:.4f}'.format(len(keep_words),
                                                   len(self.word2index),
                                                   len(keep_words)/len(self.word2index)))
        # 重新创词典
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:"PAD",SOS_token:'SOS',EOS_token:"EOS"}
        self.num_words = 3
        for word in keep_words:
            self.addWord(word)
    
# 小写并删除非字母字符
def normalizeString(seq):
    seq = seq.lower()
    seq = re.sub(r"([.!?])",r"\1",seq)
    seq = re.sub(r"[^a-zA-Z.!?]+",r" ",seq)
    return seq
    
# 使用字符串句子，返回单词索引的句子
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]
    

## 定义编码器
通过torch..nn.GRU 模块实现编码器的RNN。**本模块接收一批语句（嵌入单词的向量）的输入，他在内部遍历这些句子，每次一个标记，计算隐藏状态**。我们将这个模块初始化为双向的，**意味着我们有两个独立的GRUs；一个按时间顺序遍历序列，另一个按相反序列遍历序列**。我们最终返回这两个GRUs的和。  

由于我们的模型是使用**批处理进行训练的**，所以我们的EncoderRNN模型的forward函数需要一个填充的输入批处理。为了批量处理可变长度的句子，我们通过MAX_LENGTH令牌润徐一个句子中支持的最大长度，并且批处理中所有小于MAX_LENGTH令牌的句子都是用我们专用的PAD_token令牌填充在最后（不够长度的自动补全）。要使用带有PyTorch RNN模块的批量填充，我们必须把转发forward命令在调用 torch.nn.utils.rnn.pack_padded_sequence 和 torch.nn.utils.rnn.pad_packed_sequence 数据转换时进行打包。注意，forward函数还接受一个Input_length列表，其中包含批处理中每个句子的长度。该输入在填充时通过torch.nn.utils.rnn.pack_padded_sequence使用。  

混合前端笔记：由于编码器的转发函数forward不包含任何依赖于数据的控制流，因此我们将使用跟踪tracing将其转换为脚本模式 script mode。在跟踪模块时，我们可以保证模块定义不变。在运行评估前，我们将在文本末初始化所有模型。

In [87]:
class EncoderRNN(nn.Module):
    def __init__(self,hidden_size, embedding, n_layer=1, dropout=0):
        super(EncoderRNN,self).__init__()
        self.n_layer = n_layer
        self.hidden_size = hidden_size
        self.embedding = embedding
        
        # 初始化GRU，input_size和hidden_size参数都设置为;'hidden_size'
        # 因为我们输入的大小是一个有多个特征的词向量==hidden.size
        self.gru = nn.GRU(hidden_size,hidden_size,n_layer,dropout=(0 if n_layer == 1 else dropout),bidirectional=True)
    
    def forward(self, input_seq, input_lengths, hidden=None):
        # 将单词索引转换为向量
        embedded = self.embedding(input_seq)
        # 为RNN模块填充批次序列
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded,lengths=input_lengths)
        # 正向通过GRU
        outputs, hidden = self.gru(packed,hidden)
        # 打开填充
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        #将双向GRU的输出结果总和
        outputs = outputs[:,:,:self.hidden_size] + outputs[:,:,self.hidden_size:]
        # 返回输出以及最终的隐藏状态
        return outputs,hidden
    

## 定义解码器的注意力模块
接下来，将定义注意力模块(Attention module)。 请注意，此模块将用作解码器模型中的子模块。Luong等人考虑了各种“分数函数”score functions, 它们取当前解码器RNN输出和整个编码器输出，并返回关注点“能值”engergies。**这个关注能值张量 attention energies tensor 与编码器输出的大小相同，两者最终相乘，得到一个加权张量**，其最大值表示在特定时间步长解码的查询语句最重要的部分。

In [127]:
# Luong的注意力层
class Attention(torch.nn.Module):
    def __init__(self,method,hidden_size):
        super(Attention,self).__init__()
        self.method = method
        if self.method not in ['dot','general','concat']:
            raise ValueError(self.method, "is not an appropriate attention method")
        
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attention = torch.nn.Linear(self.hidden_size,self.hidden_size)
        elif self.method == 'concat':
            self.attention = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))
    
    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)
    
    def general_score(self,hidden, encoder_output):
        energy = self.attention(encoder_output)
        return torch.sum(hidden * energy, dim=2)
    
    def concat_score(self,hidden,encoder_output):
        # torch.cat()用于tensor拼接
        energy = self.attention(torch.cat((hidden.expend(encoder_output.size(0),-1,-1),encoder_output),2)).tanh()
        return torch.sum(self.v * energy, dim=2)
    
    def forward(self,hidden,encoder_outputs):
        # 根据给定的方法计算注意力权重（能量）
        if self.method == 'general':
            attention_energires = self.general_score(hidden,encoder_outputs)
        elif self.method == 'concat':
            attention_energires = self.concat_score(hidden,encoder_outputs)
        elif self.method == 'dot':
            attention_energires = self.dot_score(hidden,encoder_outputs)
        
        # 转置max_length和batch_size维度
        attention_energires = attention_energires.t()
        
        # 返回softmax归一化概率分数(向量化)
        return F.softmax(attention_energires,dim=1).unsqueeze(1)

## 解码器
类似EncoderRNN，我们需要使用torch.nn.GRU模块作为我们的解码器RNN。然而，这一次我们使用单向GRU。**需要注意的是，与编码器不同，我们将向解码器RNN每次提供一个单词**。我们首先得到当前单词的嵌入并应用抛出功能dropout。接下来，我们将嵌入和最后的隐藏状态转发给GRU，得到当前的GRU输出和隐藏状态。然后，我们使用Attention模块作为一个层来获取专注权重，我们将其乘以编码器的输出获得我们的参与编码器输出。我们使用这个参数编码器输出作为文本context张量，它表示一个加权和，表示编码器输出的哪些部分需要注意。在这里，我们使用线性层linear layer 和 softmax normalization 归一化来选择输出序列中的下一个单词。  

混合前端笔记与EncoderRNN类似，此模块不包含任何依赖于数据的控制流。因此，在初始化该模型并加载其参数之后，我们可以再次使用跟踪tracing 将其转换为Torch脚本。

In [108]:
class LuongAttentionDecoderRNN(nn.Module):
    def __init__(self,attention_model, embedding, hidden_size, output_size, n_layer=1, dropout=0.1):
        super(LuongAttentionDecoderRNN,self).__init__()
        
        # 保持参考
        self.attention_model = attention_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layer = n_layer
        self.dropout = dropout
        
        # 定义层
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size,hidden_size,n_layer,dropout=(0 if n_layer==1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.attention = Attention(attention_model,hidden_size)
    
    def forward(self, input_size, last_hidden, encoder_output):
        # 只执行正向的GRU
        embedded = self.embedding(input_size)
        embedded = self.embedding_dropout(embedded) # 经过dropout
        
        # 通过单向GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 通过当前GRU的输出计算注意力权重
        attention_weights = self.attention(rnn_output, encoder_output)
        # 注意力权重乘以编码器输出以获得新的上下文向量
        context = attention_weights.bmm(encoder_output.transpose(0,1))
        # 使用Luong的公式5来连接加权上下文向量和GRU输出
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output,context),1)
        concat_output = torch.tanh(self.concat(concat_input))
        # 使用Luong公式6来预测下一个单词
        output = self.out(concat_output)
        output = F.softmax(output,dim=1)
        # 返回输出和最终的隐藏状态
        return output, hidden
        

## 定义评估
### 贪婪搜索解码器
在聊天机器人教程中，我们使用 GreedySearchDecoder 模块来简化实际的解码过程。该模块将训练好的编码器和解码器模型作为属性，驱动输入语句（词索引向量）的编码过程，并一次一个词（词索引）迭代地解码输出响应序列。  

对输入序列进行编码很简单的：只需将整个序列张量及其对应的长度向量转发给编码器。 需要注意的是，这个模块一次只处理一个输入序列，而不是成批的序列。因此，当常数1用于声明张量大小时，它对应与批处理大小为1.要解码给定的解码器输出，我们必须通过解码器模型迭代地向前运行，该解码器模型输出softmax分数，改分数对应于每个单词在解码器序列中是正确的下一个单词的概率。  

**我们将decoder_input初始化为一个包含SOS_token的张量。在每次通过解码器之后，我们贪婪地将softmax概率最高的单词追加到decoded_words列表中。我们还使用这个单词作为下一个迭代地decoder_input。如果decoded_words列表的长度达到MAX_LENGTH，或者预测的单词是EOS_token，那么解码过程将终止。**  

该模块的forward方法设计到每次解码一个单词的输出序列时，遍历[0:max_length]的范围。因此，我们应该使用脚本将这个模块转换为Torch脚本。与我们可以跟踪的编码器和解码器模型不同，我们必须对GreedySearchDecoder模块进行一些必要的更改，以便在不出错的情况下初始化对象。换句话说，我们必须保确保我们的模块遵守脚本机制的规则，并且不使用Torch包含的Python子集之外的任何语言特性。  

下面图片我们在进行修改后与之前的GreedySearchDecoder的区别，其中红色为修改的部分，绿色为修改后的部分：

![figure.3](https://gitee.com/zyp521/upload_image/raw/master/Bb0ALH.png)

#### 变更事项
- nn.Moudle -> torch.jit.ScriptModule 为了在模块上使用PyTorch脚本化机制，模型从torch.jit.ScriptModule继承。
- 将decoder_n_layer追加到结构参数，这种变化源于这样一个事实，即我们传递给这个模块的编码器和解码器模型将是TraceModule(非模块)的子模块。因此，我们无法使用decoder.n_layers 访问解码器的层数，相反，我们对此进行计划，并在模块构建过程中传入此值。
- 将新属性作为常量保存在最初的实现中，我们可以在GreedySearchDecoder的forward方法中自由地使用来自全局范围的变量，然而，现在我们正在使用的脚本，我们没有这种自由，因为脚本处理的设想4是我们不一定要保留Python对象，尤其是在导出时，**一个简单的解决方案是将全局作用域中的这些值作为属性存储到构造函数中的模块中，并将他们添加到一个名为__constants__的特殊列中，以便forward方法中构造图形时将他们用作文本值**。这种做法的一个例子在19行，取代device和SOS_token全局值，我们将使用常量属性 self._device 和 self._SOS_token。
- 强制forward方法的参数类型默认情况下，Torch脚本函数的所有参数都是假定为张量。如果需要传递不同类型的参数，可以使用PEP 3107中引入的函数类型注释。此外，还尅使用MyPy-style类型的注释声明不同类型的参数。

In [130]:
class GreedySearchDecoder(nn.Module):
    def __init__(self,encoder,decoder,decoder_n_layer):
        super(GreedySearchDecoder,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.decoder_n_layer = decoder_n_layer
    
    def forward(self,sequence,length,max_length=MAX_LENGTH):
        encoder_outputs, encoder_hidden = self.encoder(sequence,length)
        decoder_hidden = encoder_hidden[:self.decoder_n_layer]
        # 初始化decoder输入
        decoder_input = torch.ones(1,1,device=device,dtype=torch.long) * SOS_token
        
        all_tokens = torch.zeros([0],device=device,dtype=torch.long)
        all_scores = torch.zeros([0],device=device)
        # 迭代进行，之前提到解码器是一个一个单词进行输入的，编码器则可以一起输入
        for i in range(max_length):
            # 正向通过解码器
            decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden,encoder_outputs)
            # 计算分数
            decoder_scores, decoder_input = torch.max(decoder_output,dim=1)
            # 记录
            all_tokens = torch.cat((all_tokens,decoder_input),dim=0)
            all_scores = torch.cat((all_scores,decoder_scores),dim=0)
            # 调整输入
            decoder_input = torch.unsqueeze(decoder_input,0) # 添加维度
        
        return all_tokens,all_scores

### 输入评估
我们定一些函数来计算输入。求值函数'evaluate'接受一个规范化字符串语句，将其处理为其对应的单词索引张量（批处理大小为1），并且将该张量传递给一个名为'searcher'和'GreedySearchDecoder'实例，已处理编码/解码过程。检索器返回输出的单词索引向量和一个分数张量，该张量对应每个编码的单词标记的'softmax'最后一步是使用'voc.index2word'将每个单词索引转换为器字符串表示的形式。  

我们还定义了两个函数来计算输入语句。'evaluateInput'函数提示用户输入，并且计算输入。他持续请求另一次输入，知道用户输入"q"或“quit”。  

'evaluateExample'函数只接受一个字符串输入语句作为参数，对其进行更规范化、计算并输出响应。

In [134]:
def evaluate(encoder,decoder,searcher,voc,sentence,max_length=MAX_LENGTH):
    # words->index
    index_batch = [indexesFromSentence(voc,sentence)]
    # 创建长度张量
    lengths = torch.tensor([len(index) for index in index_batch])
    # 转置批量的维度以匹配模型的期望
    input_batch = torch.LongTensor(index_batch).transpose(0,1)
    # 使用适当的设备
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # 用searcher解码句子
    tokens, scores = searcher(input_batch,lengths,max_length)
    # index->word
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words

# 评估来自用户的输入
def evaluateInpute(encoder,decoder,searcher,voc):
    input_sentence = ''
    while(1):
        try:
            # 获取输入的句子
            input_sentence = input('>')
            # 检查是否结束
            if input_sentence == 'q' or input_sentence == 'quit':
                break
            # 规范化句子
            input_sentence = normalizeString(input_sentence) # 正则化处理句子
            # 评估句子
            output_words = evaluatea(encoder,decoder,searcher,voc,input_sentence)
            # 格式化和打印
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:',' '.join(output_words))
        except KeyError:
            print('Error:Encountered unknown word')


# 规范化输入句子并调用evaluate（）
def evaluateExample(sentence,encoder,decoder,searcher,voc):
    # 规范化句子
    input_sentence = normalizeString(sentence)
    # 评估句子
    output_words = evaluate(encoder,decoder,searcher,voc,input_sentence)
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    print('Bot:',' '.join(output_words))

## 加载预训练参数
### 使用托管模型
- 下载模型：[click here](https://download.pytorch.org/models/tutorials/4000_checkpoint.tar)

- 设置loadFilename变量作为下载的检查点文件的路径
- 将checkpoint = torch.load(loadFilename)行取消注释，表示托管模型在GPU上训练

### 使用自己的模型
记载自己的预训练模型设计步骤：
- 将loadFilename变量设置为希望加载的检查点文件路径。注意如果你遵循各种名称的更改。
- 如果你在CPU上训练，请确保你在checkpoint=torch.load(loadFilename)行打开了检查点。如果你在GPU上训练，并且在CPU上运行这篇教程，解除checkpoint=torch.load(loadFilename,map_location=torch.devie('cpu'))的注释。

## 模型配置
我们像往常一样初始化并将参数加载到编码器和解码器中。另外，在跟踪模型之前，我们必须使用.to(device)来设置模型的设备选项，调用.eval()来设置抛出层dropout layer为test mode。

In [128]:
save_dir = os.path.join("checkpoint")
corpus_name = "cornell movie-dialogs corpus"

# 配置模型 
model_name = 'cb_name'
attention_model = 'dot' # 'general' or 'concat'
hidden_size = 500
encoder_n_layer = 2
decoder_n_layer = 2
dropout=0.1
batch_size=64

# 加载托管模型
checkpoint_name = '4000_checkpoint.tar'
loadFilename = os.path.join('checkpoint/'+checkpoint_name)

# 加载自己模型 checkpoint命名规则可自行决定
# checkpoint_iter=4000
# loadFilename = os.path.join(save_dir,model_name,corpus_name,'{}_{}-{}'.format(encoder_n_layer,decoder_n_layer,hidden_sizedden_size),'{}_checkpoint.tar'.format(checkpoint_iter))

# 加载模型
checkpoint = torch.load(loadFilename,map_location=torch.device('cpu')) # 强制CPU
encoder_dict = checkpoint['en']
decoder_dict = checkpoint['de']
encoder_optimizer_dict = checkpoint['en_opt']
decoder_optimizer_dict = checkpoint['de_opt']
embedding_dict = checkpoint['embedding']
voc = Voc(corpus_name)
voc.__dict__ = checkpoint['voc_dict']

print('Building encoder and decoder....')

# 初始化词向量
embedding = nn.Embedding(voc.num_words,hidden_size)
embedding.load_state_dict(embedding_dict)

# 初始化编码器和解码器模型
encoder = EncoderRNN(hidden_size,embedding,encoder_n_layer,dropout)
decoder = LuongAttentionDecoderRNN(attention_model,embedding,hidden_size,voc.num_words,decoder_n_layer,dropout)

# 加载训练模型参数
encoder.load_state_dict(encoder_dict)
decoder.load_state_dict(decoder_dict)

# 设置转换
encoder = encoder.to(device)
decoder = decoder.to(device)

# 将dropout层开启eval模式
encoder.eval()
decoder.eval()
print('Models built and ready to go !')

Building encoder and decoder....
Models built and ready to go !


## 模型使用
对于使用torch脚本部分，这里我们暂时先不进行了研究了。本章重点在还是对模型的研究上。为的是了解端到端模型。  

下面是我们对上述模型的使用

In [135]:
sentences = []
searcher = GreedySearchDecoder(encoder,decoder,decoder_n_layer)

while(1):
    seq = input('Your input:')
    if seq == 'q' or seq == 'quit':
        break
    evaluateExample(seq,encoder,decoder,searcher,voc)
    sentences.append(seq)

Your input:hello
Bot: hello .
Your input:hi man
Bot: hi .
Your input:what's up
Bot: i m talking about you .
Your input:q
